In [8]:
try:
    import cv2
except:
    %pip install opencv-python-headless==4.9.0.80

import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import json
import pandas as pd
from datetime import datetime
from multiprocessing import Pool, cpu_count
import shutil

print('import successfull')

import successfull


### Constants

In [9]:
BASE_PATH = Path('/home/jovyan/work/videos/')
VIDEO_PATHS = list(BASE_PATH.rglob('*.MP4')) + list(BASE_PATH.rglob('*.mp4'))
CSV_PATHS = list(BASE_PATH.rglob('*.csv'))

T1_PATHS_str = [str(file) for file in VIDEO_PATHS if not re.search('gelöschte|clipped|T2|cut|S18', str(file))]
T1_PATHS = [Path(file) for file in T1_PATHS_str]

#T2_PATHS_str = [str(file) for file in VIDEO_PATHS if not re.search('gelöschte|clipped|T1', str(file))]
#T2_PATHS = [Path(file) for file in T2_PATHS_str]


ELAN = '_ELAN'
SYNC = '_sync'

idx_category = {
    0.0: 'neutral',
    1.1: 'negative',
    1.2: 'positive'
}
category_idx = {
    'neutral': 0.0,
    'negative': 1.1, 
    'positive': 1.2
}

print(len(T1_PATHS))
#print(len(T2_PATHS))

179


In [ ]:
T1_PATHS_str.sort()
T1_PATHS_str

In [7]:
files_dict = find_files()
files_dict = get_times(files_dict)
files_dict = get_analysis(files_dict)

100%|██████████| 179/179 [00:00<00:00, 367.94it/s]

saved file into: /home/jovyan/work/output/files.json



100%|██████████| 175/175 [00:00<00:00, 1597.78it/s]


saved file into: /home/jovyan/work/output/files.json


100%|██████████| 175/175 [00:01<00:00, 106.35it/s]

saved file into: /home/jovyan/work/output/files.json


In [98]:
times_dict = {
    'START_S001_T1_La1': "16:30:56",
    'START_S001_T1_La2': "16:55:21",
    'START_S001_T1_LaCon_Ki': "17:12:57",
    'START_S001_T1_LaCon_Mu': "17:22:38",

    'START_S002_T1': "16:59:00",
    'START_S002_T1_La1': "00:00:00",
    'START_S002_T1_La2': "00:00:00",
    
    'START_S004_T1_La1': "00:00:00",
    'START_S004_T1_La2':  "00:00:00",
    'START_S004_T1_LaCon_Ki':  "00:00:00",
    'START_S004_T1_LaCon_Mu':  "00:00:00",
    
    'START_S005_T1_La1':  "17:12:00",
    'START_S005_T1_La2':  "00:00:00",
    'START_S005_T1_LaCon_Ki':  "00:00:00",
    'START_S005_T1_LaCon_Mu_cut':  "00:00:00",
    
    'START_S006_T1_La1': "15:06:05",
    'START_S006_T1_La2': "15:28:55",
    'START_S006_T1_LaCon_Ki': "15:48:04",
    'START_S006_T1_LaCon_Mu': "15:55:54",
    
    
}


In [99]:
def show_image(img):
    plt.imshow(img, cmap='gray')
    plt.show()

def save_image(path, img):
    cv2.imwrite(path, img)


In [106]:
def extract_frames(files_dict, times_dict):
    for file, file_info in tqdm(files_dict.items()):
        # init cv2 parameters
        cap = cv2.VideoCapture(str(file_info['path']))        
        fps, num_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # set output directories for saving
        output_dir = Path(f'/home/jovyan/work/output/frames/{file}')
        if not output_dir.exists():
            output_dir.mkdir(parents=True, exist_ok=True)
            
        # time stamp computations
        start_time = datetime.strptime(file_info['start_time'], '%H:%M:%f').time()
        video_time = datetime.strptime(times_dict[file], '%H:%M:%S').time()
        
        if not video_time == datetime.strptime('00:00:00', '%H:%M:%S').time():
            start_datetime = datetime.combine(datetime.today(), start_time)
            video_datetime = datetime.combine(datetime.today(), video_time)
            diff = (start_datetime - video_datetime).seconds
        else: 
            diff = 0
        start_idx = int(fps*diff)
    
        # get analysis file
        if 'analysis_file' in file_info and file_info['analysis_file']:
            df = pd.read_csv(file_info['analysis_file'])

            df = pd.read_csv(file_info['analysis_file'])
            df = df[['timestamp_start_short', 'label']]
        
            # extract frames for timestamps from analysis file
            for index, (timestamp, label) in df.iterrows():
                frame_idx = start_idx + int(timestamp * fps)
                if frame_idx >= num_frames:
                    break
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                _, frame = cap.read()
                res_path = str(output_dir / f'{file}_F{index}_T{timestamp}_L{label}.jpg')
                save_image(res_path, frame)
        else:
            pass   

# AB HIER QUATSCH

In [178]:
def process_files(video_paths=T1_PATHS):
    meta_dict = {}
    
    for video_path in tqdm(video_paths):
        name = video_path.stem
        folder = video_path.parent.parent
        
        meta_dict[str(name)] = {
            'path': str(video_path),
            'start_end_file': "",
            'sync_file': "",
            'start_time': '00:00:00',
            'end_time': '00:00:00',
            'Mu_df': '',
            'Ki_df: ''
        } 
        
        # set output path
        destination_folder = Path('/home/jovyan/work/output/') / name
        
        # search for timestamp csv & move to folder
        csvs = [str(x) for x in folder.iterdir() if x.is_file() and x.suffix == '.csv']
        match_csv = [file for file in csvs if re.search(name, file)]
        match_csv = match_csv[0] if match_csv else None

        videoanalyse_folder = Path(folder / 'Videoanalyse')
        if videoanalyse_folder.exists() and videoanalyse_folder.is_dir():
            # search for analysis txt file and move to folder
            txt_name = name[:13] + ELAN + name[13:] + SYNC
            txts = [str(x) for x in videoanalyse_folder.iterdir() if x.is_file() and x.suffix == '.txt']
            match_txt = [file for file in txts if re.search(txt_name, file)]
            match_txt = match_txt[0] if match_txt else None

        # only create output if both files exist
        if match_csv and match_txt: 
            destination_folder.mkdir(parents=True, exist_ok=True)
            
            shutil.copy(match_csv, str(destination_folder))
            meta_dict[str(name)]['start_end_file'] = str(destination_folder) + '/' + str(Path(match_csv).name)
            shutil.copy(match_txt, str(destination_folder))
            meta_dict[str(name)]['sync_file'] = str(destination_folder) + '/' + str(Path(match_txt).name)
        
        # delete if some file do not exist
        if str(name) in meta_dict and meta_dict[str(name)]['sync_file'] == "":
                del meta_dict[str(name)]
    
    # save json
    json_file = '/home/jovyan/work/output/meta_file.json'
    with open(json_file, 'w') as f:
        json.dump(meta_dict, f, indent=4)
        
    return meta_dict

SyntaxError: unterminated string literal (detected at line 15) (4091518770.py, line 15)

In [ ]:
def process_times(file_dict=file_dict):
    # adapt start and end times
    for (file, file_info) in tqdm(file_dict.items()):
        se_file = file_dict[str(file)]['start_end_file']
        if se_file:
            df = pd.read_csv(se_file)
            file_dict[str(file)]['start_time'] = f"{df.iloc[0]['hour']}:{df.iloc[0]['minute']}:{df.iloc[0]['milisecond']}"
            file_dict[str(file)]['end_time'] = f"{df.iloc[1]['hour']}:{df.iloc[1]['minute']}:{df.iloc[1]['milisecond']}"
   
    # save json
    json_file = '/home/jovyan/work/output/meta_file.json'
    with open(json_file, 'w') as f:
        json.dump(file_dict, f, indent=4)
    return file_dict

In [179]:
def process_analysis(file_dict=time_dict):
    for (file, file_info) in tqdm(file_dict.items()):
        sync_file = file_dict[str(name)]['sync_file']
        if sync_file:
            df = pd.read_csv(sync_file, sep="	", header=None)
            if(len(df.columns) >= 10):
                df = df.drop(df.columns[-1], axis=1)
            columns = ['category', 'file', 'timestamp_start_long', 
                       'timestamp_start_short', 'timestamp_end_long', 'timestamp_end_short', 
                       'length_long', 'length_short', 'label']
            df.columns = columns
            # create df for both mother and child and save to output
            mu_df = filtered_df[filtered_df['category'].str.contains('SE_Mu')].reset_index(drop=True)
            ki_df = filtered_df[filtered_df['category'].str.contains('SE_Ki')].reset_index(drop=True)

            output_dir = Path('/home/jovyan/work/output/' + file)
            output_dir.mkdir(parents=True, exist_ok=True)
            
            mu_path = str(output_dir) + '/MU_DF.csv'
            ki_path = str(output_dir) + '/MI_DF.csv'
            
            mu_df.to_csv(mu_path, index=False)
            ki_df.to_csv(ki_path, index=False)    

            file_dict[str(file)]['Mu_df'] = mu_path
            file_dict[str(file)]['Ki_df'] = ki_path

        # save json
        json_file = '/home/jovyan/work/output/meta_file.json'
        with open(json_file, 'w') as f:
            json.dump(file_dict, f, indent=4)
        return file_dict

In [190]:
def move_videos(file_dict=file_dict):
    for (file, file_info) in tqdm(file_dict.items()):
        
        video_path = file_dict[str(file)]['path']
        destination_folder = Path('/home/jovyan/work/output/') / Path(file).name
        
        shutil.copy(video_path, str(destination_folder))

        file_dict[str(file)]['path'] = str(destination_folder / Path(video_path).name))
        
        # save json
    json_file = '/home/jovyan/work/output/meta_file.json'
    with open(json_file, 'w') as f:
        json.dump(file_dict, f, indent=4)
    return file_dict

SyntaxError: unmatched ')' (4129195882.py, line 9)

In [174]:
file_dict = process_files()

100%|██████████| 179/179 [00:18<00:00,  9.94it/s]


In [175]:
file_dict = process_times()

100%|██████████| 87/87 [00:00<00:00, 232.37it/s]


In [176]:
process_analysis()

100%|██████████| 87/87 [00:01<00:00, 52.06it/s]


In [191]:
move_videos()

  0%|          | 0/87 [00:00<?, ?it/s]

/home/jovyan/work/output/START_S001_T1_La1/START_S001_T1_La1.MP4


In [ ]:
def extract_frames(files_dict=time, times_dict):
    for (file, file_info) in tqdm(files_dict.items()):
        # init cv2 parameters
        cap = cv2.VideoCapture(str(file_info['path']))        
        fps, num_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # set output directories for saving
        output_dir = Path(f'/home/jovyan/work/output/frames/{file}')
        if not output_dir.exists():
            output_dir.mkdir(parents=True, exist_ok=True)
            
        # time stamp computations
        start_time = datetime.strptime(file_info['start_time'], '%H:%M:%f').time()
        video_time = datetime.strptime(times_dict[file], '%H:%M:%S').time()
        
        if not video_time == datetime.strptime('00:00:00', '%H:%M:%S').time():
            start_datetime = datetime.combine(datetime.today(), start_time)
            video_datetime = datetime.combine(datetime.today(), video_time)
            diff = (start_datetime - video_datetime).seconds
        else: 
            diff = 0
        start_idx = int(fps*diff)
    
        # get analysis file
        if 'analysis_file' in file_info and file_info['analysis_file']:
            df = pd.read_csv(file_info['analysis_file'])

            df = pd.read_csv(file_info['analysis_file'])
            df = df[['timestamp_start_short', 'label']]
        
            # extract frames for timestamps from analysis file
            for index, (timestamp, label) in df.iterrows():
                frame_idx = start_idx + int(timestamp * fps)
                if frame_idx >= num_frames:
                    break
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                _, frame = cap.read()
                res_path = str(output_dir / f'{file}_F{index}_T{timestamp}_L{label}.jpg')
                save_image(res_path, frame)
        else:
            pass   